In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/dogecoin-historical-data/DOGE-USD.csv")
df.head()

In [ ]:
df.corr()

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'], infer_datetime_format=True)
df.set_index('Date',inplace=True)
df.head()

In [ ]:
df.dropna(inplace=True)
df.info()

In [ ]:
# Close column is our y feature (predicted feature). So we should make feature engineering to get higher corr values with y 
# We will take corr values between 0.8-0.2 due to Pearson Correlation Matrix but we can take Volume because its at the borderline
df["gap"] = (df["High"] - df["Low"]) * df["Volume"]
df["y"] = df["High"] / df["Volume"]
df["z"] = df["Low"] / df["Volume"]
df["a"] = df["High"] / df["Low"]
df["b"] = (df["High"] / df["Low"]) * df["Volume"]
abs(df.corr()["Close"].sort_values(ascending=False))

In [ ]:
df = df[["Close","Volume","gap","a","b"]]
df.head()

In [ ]:
#!pip install pycaret 
from pycaret.regression import *
exp_name = setup(data=df,target="Close")
best_model = compare_models()

In [ ]:
model1 = create_model("gbr")

In [ ]:
tuned_model = tune_model(model1)

## Let's make Time Series Analysis too!

In [ ]:
## I'm gonna analyze last 30 days because significant changes have occured in these 30 days 
df2 = df.tail(30)
df2

In [ ]:
#!pip install pmdarima
from pmdarima.arima import auto_arima

model = auto_arima(df2["Close"],df2.drop("Close",axis=1),
                      seasonal=True, m=1,
                      start_d=0, D=None, 
                      start_q=0, start_p=0,
                      trace=True,
                      error_action='ignore',test="adf",
                      suppress_warnings=True)

print(model.summary())


In [ ]:
df2.info()

In [ ]:
train= df2[:11]
test= df2[-19:]

print(train.shape, test.shape)
train

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model= SARIMAX(endog=train["Close"],exog = train.drop("Close",axis=1), order=(2,1,1))
results=model.fit()
print(results.summary())

In [ ]:
start= 11
end= 29
predictions= results.predict(start=start, end=end,exog = test.drop("Close",axis=1))
predictions

In [ ]:
test["Close"].plot(legend=True,figsize=(12,6))
predictions.plot(label='TimeSeries',legend=True);

## Well trends looks like same so I think there are similar speculations on the market compared to last month.

## Thanks for reviewing, feel free to comment!